<a href="https://colab.research.google.com/github/charlie-aashutosh/Tensorflow-2.0/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 25kB/s 
     |████████████████████████████████| 419kB 38.1MB/s 
     |████████████████████████████████| 3.0MB 37.7MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 


In [1]:
!pip install tqdm

In [2]:
#downloading Cats vs Dogs classifier dataset

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-08-10 17:59:58--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 173.194.197.128, 64.233.191.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   119MB/s    in 0.6s    

2020-08-10 17:59:59 (119 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [4]:
#Data preprocessing and importing dependencies

In [5]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.3.0'

In [6]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [7]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [8]:
zip_object.extractall("./")

In [9]:
zip_object.close()

In [10]:
#setting dataset path
dataset_path_new = "./cats_and_dogs_filtered/"

In [11]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [12]:
IMG_SHAPE = (128, 128, 3)

In [13]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [14]:
# base_model.summary()

### Freezing the base model

In [15]:
base_model.trainable = False

### Defining the custom head for our network

In [16]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [17]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [18]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [19]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [20]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [21]:
# model.summary()

### Compiling the model

In [22]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [23]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [24]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [25]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Training the model

In [26]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
16/16 [==============================] - 41s 3s/step - loss: 0.6355 - accuracy: 0.6530 - val_loss: 0.5962 - val_accuracy: 0.6840
Epoch 2/5
16/16 [==============================] - 40s 2s/step - loss: 0.5301 - accuracy: 0.7530 - val_loss: 0.5149 - val_accuracy: 0.7640
Epoch 3/5
16/16 [==============================] - 41s 3s/step - loss: 0.4603 - accuracy: 0.8110 - val_loss: 0.4518 - val_accuracy: 0.8020
Epoch 4/5
16/16 [==============================] - 41s 3s/step - loss: 0.4046 - accuracy: 0.8495 - val_loss: 0.3999 - val_accuracy: 0.8480
Epoch 5/5
16/16 [==============================] - 40s 3s/step - loss: 0.3592 - accuracy: 0.8780 - val_loss: 0.3568 - val_accuracy: 0.8660


### Transfer learning model evaluation

In [27]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [28]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8659999966621399
